<a href="https://colab.research.google.com/github/gapple95/machine_learning/blob/master/colab_image_classifier_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import itertools
import os

import matplotlib.pylab as plt
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub

import time as t #시간 측정용

print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")
print("GPU name : ",tf.test.gpu_device_name())

TF version: 2.3.0
Hub version: 0.9.0
GPU is available
GPU name :  /device:GPU:0


설정

In [12]:
module_selection = ("mobilenet_v2_100_224", 224) 
handle_base, pixels = module_selection
MODULE_HANDLE ="https://tfhub.dev/google/imagenet/{}/feature_vector/4".format(handle_base)
IMAGE_SIZE = (pixels, pixels)
print("Using {} with input size {}".format(MODULE_HANDLE, IMAGE_SIZE))

BATCH_SIZE = 32 

Using https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4 with input size (224, 224)


TF2 모델 모듈 갖고오기

In [15]:
data_dir = tf.keras.utils.get_file(
    'flower_photos',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    untar=True)

In [14]:
datagen_kwargs = dict(rescale=1./255, validation_split=.20)
dataflow_kwargs = dict(target_size=IMAGE_SIZE, batch_size=BATCH_SIZE,
                   interpolation="bilinear")

valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    **datagen_kwargs)
valid_generator = valid_datagen.flow_from_directory(
    data_dir, subset="validation", shuffle=False, **dataflow_kwargs)

do_data_augmentation = False 
if do_data_augmentation:
  train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
      rotation_range=40,
      horizontal_flip=True,
      width_shift_range=0.2, height_shift_range=0.2,
      shear_range=0.2, zoom_range=0.2,
      **datagen_kwargs)
else:
  train_datagen = valid_datagen
train_generator = train_datagen.flow_from_directory(
    data_dir, subset="training", shuffle=True, **dataflow_kwargs)

Found 731 images belonging to 5 classes.
Found 2939 images belonging to 5 classes.


데이터 갖고오기

In [16]:
do_fine_tuning = False 
print("Building model with", MODULE_HANDLE)
model = tf.keras.Sequential([
    # Explicitly define the input shape so the model can be properly
    # loaded by the TFLiteConverter
    tf.keras.layers.InputLayer(input_shape=IMAGE_SIZE + (3,)),
    hub.KerasLayer(MODULE_HANDLE, trainable=do_fine_tuning),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(train_generator.num_classes,
                          kernel_regularizer=tf.keras.regularizers.l2(0.0001))
])
model.build((None,)+IMAGE_SIZE+(3,))
model.summary()

Building model with https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 1280)              2257984   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 6405      
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________


모델 정의

In [17]:
with tf.device('/device:GPU:0'):
  model.compile(
    optimizer=tf.keras.optimizers.SGD(lr=0.005, momentum=0.9), 
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
    metrics=['accuracy'])

  steps_per_epoch = train_generator.samples // train_generator.batch_size
  validation_steps = valid_generator.samples // valid_generator.batch_size
  hist = model.fit(
      train_generator,
      epochs=5, steps_per_epoch=steps_per_epoch,  
      validation_data=valid_generator,
      validation_steps=validation_steps).history

Epoch 1/5
91/91 [==============================] - 17s 187ms/step - loss: 0.9429 - accuracy: 0.7489 - val_loss: 0.7484 - val_accuracy: 0.8551
Epoch 2/5
91/91 [==============================] - 16s 180ms/step - loss: 0.7106 - accuracy: 0.8696 - val_loss: 0.6987 - val_accuracy: 0.8665
Epoch 3/5
91/91 [==============================] - 16s 181ms/step - loss: 0.6467 - accuracy: 0.9040 - val_loss: 0.7032 - val_accuracy: 0.8636
Epoch 4/5
91/91 [==============================] - 16s 180ms/step - loss: 0.6242 - accuracy: 0.9157 - val_loss: 0.6843 - val_accuracy: 0.8778
Epoch 5/5
91/91 [==============================] - 17s 183ms/step - loss: 0.6070 - accuracy: 0.9284 - val_loss: 0.6883 - val_accuracy: 0.8793


In [ ]:
model.compile(
  optimizer=tf.keras.optimizers.SGD(lr=0.005, momentum=0.9), 
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
  metrics=['accuracy'])

steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = valid_generator.samples // valid_generator.batch_size
hist = model.fit(
    train_generator,
    epochs=5, steps_per_epoch=steps_per_epoch,  
    validation_data=valid_generator,
    validation_steps=validation_steps).history

모델 훈련